In [1]:
import os, tqdm, json, pickle, gc, zipfile, itertools, time, collections
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

def get_responses(PATH_TO_RESPONSES):
    L_FILENAMES = os.listdir(PATH_TO_RESPONSES)
    L_RESPONSES = []
    for fnm in tqdm.tqdm(L_FILENAMES):
        try:
            pth = os.path.join(PATH_TO_RESPONSES, fnm)
            with open(pth, 'r') as f:
                d_rsp = json.load(f)
            L_RESPONSES.append(d_rsp)
        except:
            pass
    idx_ordered = np.argsort([d_game['id'] for d_game in L_RESPONSES])[::-1]
    L_RESPONSES = np.array(L_RESPONSES)[idx_ordered].tolist()
    return L_RESPONSES

def get_game_profile(L_RESPONSES):

    L_games = []

    L_PLAYER_STAT_KEYS = [
        'adr', 'assists', 'deaths', 'first_kills_diff', 'flash_assists', 
        'headshots', 'k_d_diff', 'kast', 'kills', 'rating'
    ]

    for rsp in tqdm.tqdm(L_RESPONSES):
        
        try:

            d = {}
            d['id'] = rsp['id']

            date = parser.parse(rsp['begin_at'])
            d['timestamp'] = date.timestamp()
            d['year'] = date.year
            d['month'] = date.month
            d['day'] = date.day
            d['weekday'] = date.weekday()
            d['hour'] = date.hour

            d['map__id'] = rsp['map']['id']

            d['league__id'] = rsp['match']['league']['id']
            d['serie__id'] = rsp['match']['serie']['id']
            d['tournament__id'] = rsp['match']['tournament']['id']
            try:
                d['serie__tier'] = str.lower(rsp['match']['serie']['tier']).strip()
            except:
                d['serie__tier'] = 'default'
            d['tournament__name'] = str.lower(rsp['match']['tournament']['name']).strip()
            pp = rsp['match']['tournament']['prizepool']
            try:
                d['tournament__prizepool_currency'] = str.lower(''.join(pp.split(' ')[1:]))
                d['tournament__prizepool_size'] = int(pp.split(' ')[0])
            except:
                d['tournament__prizepool_currency'] = 'default'
                d['tournament__prizepool_size'] = 0

            df_rounds = pd.DataFrame.from_records(rsp['rounds'])            

            r1_winner_id = df_rounds.query('round==1')['winner_team'].iloc[0]
            r2_winner_id = df_rounds.query('round==2')['winner_team'].iloc[0]
            r16_winner_id = df_rounds.query('round==16')['winner_team'].iloc[0]
            r17_winner_id = df_rounds.query('round==17')['winner_team'].iloc[0]
            d_h1_win_count = df_rounds.query('round<=15')['winner_team'].value_counts().to_dict()
            d_h2_win_count = df_rounds.query('round>15')['winner_team'].value_counts().to_dict()
            d_h1_outcome_count = df_rounds.query('round<=15')['outcome'].value_counts().to_dict()
            d_h2_outcome_count = df_rounds.query('round>15')['outcome'].value_counts().to_dict()


            d['team1__id'] =  df_rounds.query('round==1')['ct'].iloc[0]
            d['team2__id'] =  df_rounds.query('round==1')['terrorists'].iloc[0]

            d['team1__win'] = int(df_rounds['winner_team'].value_counts().idxmax() == d['team1__id'])
            d['team1__r1_win'] = 1 if r1_winner_id==d['team1__id'] else 0
            d['team1__r2_win'] = 1 if r2_winner_id==d['team1__id'] else 0
            d['team1__r16_win'] = 1 if r16_winner_id==d['team1__id'] else 0
            d['team1__r17_win'] = 1 if r17_winner_id==d['team1__id'] else 0
            d['team1__h1_win_count'] = d_h1_win_count[d['team1__id']] if d['team1__id'] in d_h1_win_count.keys() else 0
            d['team1__h2_win_count'] = d_h2_win_count[d['team1__id']] if d['team1__id'] in d_h2_win_count.keys() else 0
            for k, v in d_h1_outcome_count.items():
                d[f'team1__h1_{k}_count'] = v
            for k, v in d_h2_outcome_count.items():
                d[f'team1__h2_{k}_count'] = v
            
            if len(rsp['players'])==10:
                counter4team1, counter4team2 = 1, 6
                for p in rsp['players']:
                    
                    team_id = p['team']['id']

                    if team_id == d['team1__id']: 
                        
                        d[f'player{counter4team1}__id'] = p['player']['id']
                        try:
                            d[f'player{counter4team1}__age'] = int(p['player']['age'])
                        except:
                            d[f'player{counter4team1}__age'] = -999
                        try:
                            d[f'player{counter4team1}__hometown'] = str.lower(p['player']['hometown']).strip()
                        except:
                            d[f'player{counter4team1}__hometown'] = 'default'
                        try:
                            d[f'player{counter4team1}__nationality'] = str.lower(p['player']['nationality']).strip()
                        except:
                            d[f'player{counter4team1}__nationality'] = 'default'
                        try:
                            d['team1__location'] = str.lower(p['team']['location']).strip()
                        except:
                            d['team1__location'] = 'default'

                        for key in L_PLAYER_STAT_KEYS:
                            value = p[key]
                            d[f'player{counter4team1}__{key}'] = value if pd.notnull(value) else 0.0

                        counter4team1+=1

                    else:

                        d[f'player{counter4team2}__id'] = p['player']['id']
                        try:
                            d[f'player{counter4team2}__age'] = int(p['player']['age'])
                        except:
                            d[f'player{counter4team2}__age'] = -999
                        try:
                            d[f'player{counter4team2}__hometown'] = str.lower(p['player']['hometown']).strip()
                        except:
                            d[f'player{counter4team2}__hometown'] = 'default'
                        try:
                            d[f'player{counter4team2}__nationality'] = str.lower(p['player']['nationality']).strip()
                        except:
                            d[f'player{counter4team2}__nationality'] = 'default'
                        try:
                            d['team2__location'] = str.lower(p['team']['location']).strip()
                        except:
                            d['team2__location'] = 'default'

                        for key in L_PLAYER_STAT_KEYS:
                            value = p[key]
                            d[f'player{counter4team2}__{key}'] = value if pd.notnull(value) else 0.0

                        counter4team2+=1

                ser = pd.Series(d)                

                cs4p = []
                for i in range(1, 11):
                    cs4p.extend(ser.index[ser.index.str.contains(f'player{i}__')].tolist())
                
                d = pd.concat([ser.drop(cs4p), ser[cs4p]]).to_dict()

                L_games.append(d)            
            
        except:
            pass

    df_games = pd.DataFrame.from_records(L_games).sort_values('timestamp').reset_index(drop = True)
    del L_games
    gc.collect()

    L_PLAYER_STAT_KEYS = [
        'adr', 'assists', 'deaths', 'first_kills_diff', 'flash_assists', 
        'headshots', 'k_d_diff', 'kast', 'kills', 'rating'
    ]

    for key in L_PLAYER_STAT_KEYS:
        fs = df_games.columns[df_games.columns.str.contains(key)]
        df_games[fs] = df_games[fs].fillna(0.0)

    L_TEAM_STAT_KEYS = [
        
        'team1__h1_eliminated_count', 'team1__h1_defused_count',
        'team1__h1_exploded_count', 'team1__h1_timeout_count',
        'team1__h2_eliminated_count', 'team1__h2_defused_count',
        'team1__h2_exploded_count', 'team1__h2_timeout_count'
        
    ]
    df_games[L_TEAM_STAT_KEYS] = df_games[L_TEAM_STAT_KEYS].fillna(0.0)

    team_keys = df_games.columns[df_games.columns.str.contains('team1')].tolist()
           
    player_keys = []
    for i in range(1, 11):
        player_keys.extend(df_games.columns[df_games.columns.str.contains(f'player{i}__')].tolist())      

    df_games = pd.concat([df_games.drop(team_keys+player_keys, 1), df_games[team_keys], df_games[player_keys]], 1)
    df_games = df_games.dropna(subset = ['team1__id', 'team2__id']).reset_index(drop = True)    
        
    return df_games

In [2]:
# # директория с респонсами
# PATH_TO_RESPONSES = 'responses'

# L_RESPONSES = get_responses(PATH_TO_RESPONSES)
# with open('L_RESPONSES.pickle','wb') as f:
#     pickle.dump(L_RESPONSES)
with open('L_RESPONSES.pickle','rb') as f:
    L_RESPONSES = pickle.load(f)

# df_game_profile = get_game_profile(L_RESPONSES)
with open('df_game_profile_18052022_.pickle','rb') as f:
    df_game_profile = pickle.load(f)
    

In [3]:
game_info_keys = df_game_profile.columns[~df_game_profile.columns.str.contains('player|team')]
game_info_keys

Index(['id', 'timestamp', 'year', 'month', 'day', 'weekday', 'hour', 'map__id',
       'league__id', 'serie__id', 'tournament__id', 'serie__tier',
       'tournament__name', 'tournament__prizepool_currency',
       'tournament__prizepool_size'],
      dtype='object')

In [4]:
df_game_profile[game_info_keys].nunique().sort_values(ascending = False)

id                                39131
timestamp                         38687
tournament__id                     2291
serie__id                          1159
league__id                          257
tournament__prizepool_size          195
tournament__name                    151
day                                  31
tournament__prizepool_currency       25
hour                                 24
map__id                              13
month                                12
year                                  7
weekday                               7
serie__tier                           7
dtype: int64

In [5]:
player_keys = df_game_profile.columns[df_game_profile.columns.str.contains('player1__')]

In [6]:
player_keys

Index(['player1__id', 'player1__age', 'player1__hometown',
       'player1__nationality', 'player1__adr', 'player1__assists',
       'player1__deaths', 'player1__first_kills_diff',
       'player1__flash_assists', 'player1__headshots', 'player1__k_d_diff',
       'player1__kast', 'player1__kills', 'player1__rating'],
      dtype='object')

In [7]:
team1_keys = df_game_profile.columns[df_game_profile.columns.str.contains('team1__')]
team2_keys = df_game_profile.columns[df_game_profile.columns.str.contains('team2__')]

In [8]:
assert (len(game_info_keys)+len(team1_keys)+len(team2_keys)+ len(player_keys)*10)==df_game_profile.shape[1]

In [9]:
df_game_profile.shape

(39131, 174)

In [12]:
df_game_profile.head()

,id,timestamp,year,month,day,weekday,hour,map__id,league__id,serie__id,...,player10__adr,player10__assists,player10__deaths,player10__first_kills_diff,player10__flash_assists,player10__headshots,player10__k_d_diff,player10__kast,player10__kills,player10__rating
0,1396,1.452687e+09,2016,1,13,2,12,1,4164,1593,...,120.1,3.0,13,-1.0,0.0,13.0,10.0,85.7,23,1.79
1,1397,1.452690e+09,2016,1,13,2,12,3,4164,1593,...,76.9,1.0,18,0.0,0.0,11.0,-1.0,52.2,17,0.98
2,1393,1.452696e+09,2016,1,13,2,14,3,4164,1593,...,77.6,4.0,22,1.0,0.0,5.0,-10.0,56.0,12,0.70
3,1394,1.452699e+09,2016,1,13,2,15,2,4164,1593,...,113.1,6.0,16,5.0,0.0,14.0,9.0,82.6,25,1.76
4,1387,1.452704e+09,2016,1,13,2,17,2,4164,1593,...,81.3,2.0,14,0.0,0.0,6.0,3.0,77.3,17,1.24


In [15]:
df_game_profile['team1__win'].value_counts(normalize = True)

0    0.518617
1    0.481383
Name: team1__win, dtype: float64

In [16]:
TEST_SIZE = .1

df_game_profile_train, df_game_profile_test = train_test_split(df_game_profile, test_size = TEST_SIZE, shuffle = False)

In [17]:
to_enc_v1 = [
    'year', 'month', 'day', 'weekday', 'hour', 'map__id',
    'league__id', 'serie__id', 'tournament__id', 'serie__tier',
    'tournament__name', 'tournament__prizepool_currency',
    'tournament__prizepool_size'
]

In [25]:
FILL_VALUE=-999
encoder = LabelEncoder()

d_label_encoders ={}
for key in to_enc_v1:

    ser_tr = df_game_profile_train[key]
    unique_values = np.unique(ser_tr)
    ser_te = df_game_profile_test[key]
    ser_te = ser_te.apply(lambda x:x if x in unique_values else FILL_VALUE)

    encoder.fit(ser_tr)
    encoder.classes_= np.append(encoder.classes_, FILL_VALUE)

    d_label_encoders[key]=encoder

    df_game_profile_train[key] = encoder.transform(ser_tr)
    df_game_profile_test[key] = encoder.transform(ser_te)




In [34]:
team_loc = ['team2__location', 'team1__location']
n_rows_tr, n_rows_te = df_game_profile_train.shape[0], df_game_profile_test.shape[0]

unique_values = np.unique(df_game_profile_train[team_loc].values.flatten())
df_game_profile_test[team_loc] = df_game_profile_test[team_loc].applymap(lambda x:x if x in unique_values else FILL_VALUE)

encoder.fit(df_game_profile_train[team_loc].values.flatten().reshape(-1,1))
encoder.classes_= np.append(encoder.classes_, FILL_VALUE)

d_label_encoders['location']=encoder

df_game_profile_train[team_loc] = encoder.transform(df_game_profile_train[team_loc].values.flatten().reshape(-1,1)).reshape(n_rows_tr, len(team_loc))
df_game_profile_test[team_loc] = encoder.transform(df_game_profile_test[team_loc].values.flatten().reshape(-1,1)).reshape(n_rows_te, len(team_loc))

In [41]:
hometown_keys = ['player1__hometown',
  'player2__hometown',
  'player3__hometown',
  'player4__hometown',
  'player5__hometown',
  'player6__hometown',
  'player7__hometown',
  'player8__hometown',
  'player9__hometown',
  'player10__hometown']
  
nationality_keys =  ['player1__nationality',
  'player2__nationality',
  'player3__nationality',
  'player4__nationality',
  'player5__nationality',
  'player6__nationality',
  'player7__nationality',
  'player8__nationality',
  'player9__nationality',
  'player10__nationality']

n_rows_tr, n_rows_te = df_game_profile_train.shape[0], df_game_profile_test.shape[0]

unique_values = np.unique(df_game_profile_train[hometown_keys].values.flatten())
df_game_profile_test[hometown_keys] = df_game_profile_test[hometown_keys].applymap(lambda x:x if x in unique_values else FILL_VALUE)

encoder.fit(df_game_profile_train[hometown_keys].values.flatten().reshape(-1,1))
encoder.classes_= np.append(encoder.classes_, FILL_VALUE)

d_label_encoders['hometown']=encoder

df_game_profile_train[hometown_keys] = encoder.transform(df_game_profile_train[hometown_keys].values.flatten().reshape(-1,1)).reshape(n_rows_tr, len(hometown_keys))
df_game_profile_test[hometown_keys] = encoder.transform(df_game_profile_test[hometown_keys].values.flatten().reshape(-1,1)).reshape(n_rows_te, len(hometown_keys))

unique_values = np.unique(df_game_profile_train[nationality_keys].values.flatten())
df_game_profile_test[nationality_keys] = df_game_profile_test[nationality_keys].applymap(lambda x:x if x in unique_values else FILL_VALUE)

encoder.fit(df_game_profile_train[nationality_keys].values.flatten().reshape(-1,1))
encoder.classes_= np.append(encoder.classes_, FILL_VALUE)

d_label_encoders['nationality']=encoder

df_game_profile_train[nationality_keys] = encoder.transform(df_game_profile_train[nationality_keys].values.flatten().reshape(-1,1)).reshape(n_rows_tr, len(nationality_keys))
df_game_profile_test[nationality_keys] = encoder.transform(df_game_profile_test[nationality_keys].values.flatten().reshape(-1,1)).reshape(n_rows_te, len(nationality_keys))



In [49]:
player_id_keys = ['player1__id',
  'player2__id',
  'player3__id',
  'player4__id',
  'player5__id',
  'player6__id',
  'player7__id',
  'player8__id',
  'player9__id',
  'player10__id']
  
team_id_keys =  ['team1__id','team2__id']

n_rows_tr, n_rows_te = df_game_profile_train.shape[0], df_game_profile_test.shape[0]

unique_values = np.unique(df_game_profile_train[player_id_keys].values.flatten())
df_game_profile_test[player_id_keys] = df_game_profile_test[player_id_keys].applymap(lambda x:x if x in unique_values else FILL_VALUE)

encoder.fit(df_game_profile_train[player_id_keys].values.flatten().reshape(-1,1))
encoder.classes_= np.append(encoder.classes_, FILL_VALUE)

d_label_encoders['player_id']=encoder

df_game_profile_train[player_id_keys] = encoder.transform(df_game_profile_train[player_id_keys].values.flatten().reshape(-1,1)).reshape(n_rows_tr, len(player_id_keys))
df_game_profile_test[player_id_keys] = encoder.transform(df_game_profile_test[player_id_keys].values.flatten().reshape(-1,1)).reshape(n_rows_te, len(player_id_keys))

unique_values = np.unique(df_game_profile_train[team_id_keys].values.flatten())
df_game_profile_test[team_id_keys] = df_game_profile_test[team_id_keys].applymap(lambda x:x if x in unique_values else FILL_VALUE)

encoder.fit(df_game_profile_train[team_id_keys].values.flatten().reshape(-1,1))
encoder.classes_= np.append(encoder.classes_, FILL_VALUE)

d_label_encoders['team_id']=encoder

df_game_profile_train[team_id_keys] = encoder.transform(df_game_profile_train[team_id_keys].values.flatten().reshape(-1,1)).reshape(n_rows_tr, len(team_id_keys))
df_game_profile_test[team_id_keys] = encoder.transform(df_game_profile_test[team_id_keys].values.flatten().reshape(-1,1)).reshape(n_rows_te, len(team_id_keys))

In [45]:
from sklearn.base import BaseEstimator, TransformerMixin

class InRangeScaler(BaseEstimator, TransformerMixin):

    def __init__(self, out_range):
        self.out_range = out_range        
    
    def fit(self, X_tr, y_tr = None):
        assert isinstance(X_tr, pd.Series)
        self.domain = np.min(X_tr), np.max(X_tr)
        return self

    def transform(self, X_tr):
        y = (X_tr - (self.domain[1] + self.domain[0]) / 2) / (self.domain[1] - self.domain[0])
        return y * (self.out_range[1] - self.out_range[0]) + (self.out_range[1] + self.out_range[0]) / 2

rng_scaler = InRangeScaler(out_range = (.1, .9))
rng_scaler.fit(df_game_profile_train['timestamp'])

df_game_profile_train['timestamp'] = rng_scaler.transform(df_game_profile_train['timestamp'])
df_game_profile_test['timestamp'] = rng_scaler.transform(df_game_profile_test['timestamp'])

In [50]:
df_game_profile_train.head()

,id,timestamp,year,month,day,weekday,hour,map__id,league__id,serie__id,...,player10__adr,player10__assists,player10__deaths,player10__first_kills_diff,player10__flash_assists,player10__headshots,player10__k_d_diff,player10__kast,player10__kills,player10__rating
0,1396,0.100000,0,0,12,2,12,0,10,38,...,120.1,3.0,13,-1.0,0.0,13.0,10.0,85.7,23,1.79
1,1397,0.100011,0,0,12,2,12,2,10,38,...,76.9,1.0,18,0.0,0.0,11.0,-1.0,52.2,17,0.98
2,1393,0.100039,0,0,12,2,14,2,10,38,...,77.6,4.0,22,1.0,0.0,5.0,-10.0,56.0,12,0.70
3,1394,0.100052,0,0,12,2,15,1,10,38,...,113.1,6.0,16,5.0,0.0,14.0,9.0,82.6,25,1.76
4,1387,0.100073,0,0,12,2,17,1,10,38,...,81.3,2.0,14,0.0,0.0,6.0,3.0,77.3,17,1.24


In [56]:
scaler = MinMaxScaler()

team_num_keys = ['team1__h1_win_count', 'team1__h2_win_count',
       'team1__h1_eliminated_count', 'team1__h1_defused_count',
       'team1__h1_exploded_count', 'team1__h2_eliminated_count',
       'team1__h2_defused_count', 'team1__h2_exploded_count',
       'team1__h2_timeout_count', 'team1__h1_timeout_count']
       
scaler.fit(df_game_profile_train[team_num_keys])
df_game_profile_train[team_num_keys] = scaler.transform(df_game_profile_train[team_num_keys])
df_game_profile_test[team_num_keys] = scaler.transform(df_game_profile_test[team_num_keys])

In [63]:
L_PLAYER_STAT_KEYS = [
        'adr', 'assists', 'deaths', 'first_kills_diff', 'flash_assists', 
        'headshots', 'k_d_diff', 'kast', 'kills', 'rating'
]

for i in range(1, 11):
    cs = [f'player{i}__{k}' for k in L_PLAYER_STAT_KEYS]
    
    scaler.fit(df_game_profile_train[cs])
    df_game_profile_train[cs] = scaler.transform(df_game_profile_train[cs])
    df_game_profile_test[cs] = scaler.transform(df_game_profile_test[cs])

In [64]:
df_game_profile_train.head()

,id,timestamp,year,month,day,weekday,hour,map__id,league__id,serie__id,...,player10__adr,player10__assists,player10__deaths,player10__first_kills_diff,player10__flash_assists,player10__headshots,player10__k_d_diff,player10__kast,player10__kills,player10__rating
0,1396,0.100000,0,0,12,2,12,0,10,38,...,0.680068,0.130435,0.236364,0.44,0.0,0.333333,0.666667,0.857,0.164286,0.594684
1,1397,0.100011,0,0,12,2,12,2,10,38,...,0.435447,0.043478,0.327273,0.48,0.0,0.282051,0.437500,0.522,0.121429,0.325581
2,1393,0.100039,0,0,12,2,14,2,10,38,...,0.439411,0.173913,0.400000,0.52,0.0,0.128205,0.250000,0.560,0.085714,0.232558
3,1394,0.100052,0,0,12,2,15,1,10,38,...,0.640430,0.260870,0.290909,0.68,0.0,0.358974,0.645833,0.826,0.178571,0.584718
4,1387,0.100073,0,0,12,2,17,1,10,38,...,0.460362,0.086957,0.254545,0.48,0.0,0.153846,0.520833,0.773,0.121429,0.411960


In [67]:
df_game_profile_train.min().min(), df_game_profile_test.min().min()

(-999.0, -999.0)

In [70]:
df_game_profile_train = df_game_profile_train.replace({-999:0})
df_game_profile_test = df_game_profile_test.replace({-999:0})

In [73]:
df_game_profile_train.max().sort_values(ascending = False)

id                           64258.0
player5__id                   4134.0
player10__id                  4134.0
player9__id                   4130.0
player4__id                   4130.0
                              ...   
player6__first_kills_diff        1.0
player9__rating                  1.0
player6__adr                     1.0
player7__first_kills_diff        1.0
timestamp                        0.9
Length: 174, dtype: float64

In [75]:
df_game_profile_train = df_game_profile_train.convert_dtypes()
df_game_profile_test = df_game_profile_test.convert_dtypes()

In [78]:
int_cs = df_game_profile_train.select_dtypes('int').columns
df_game_profile_train[int_cs] = df_game_profile_train[int_cs].astype(int)
df_game_profile_test[int_cs] = df_game_profile_test[int_cs].astype(int)

float_cs = df_game_profile_train.select_dtypes('float').columns
df_game_profile_train[float_cs] = df_game_profile_train[float_cs].astype(float)
df_game_profile_test[float_cs] = df_game_profile_test[float_cs].astype(float)

In [79]:
df_game_profile_train.head()

,id,timestamp,year,month,day,weekday,hour,map__id,league__id,serie__id,...,player10__adr,player10__assists,player10__deaths,player10__first_kills_diff,player10__flash_assists,player10__headshots,player10__k_d_diff,player10__kast,player10__kills,player10__rating
0,1396,0.100000,0,0,12,2,12,0,10,38,...,0.680068,0.130435,0.236364,0.44,0.0,0.333333,0.666667,0.857,0.164286,0.594684
1,1397,0.100011,0,0,12,2,12,2,10,38,...,0.435447,0.043478,0.327273,0.48,0.0,0.282051,0.437500,0.522,0.121429,0.325581
2,1393,0.100039,0,0,12,2,14,2,10,38,...,0.439411,0.173913,0.400000,0.52,0.0,0.128205,0.250000,0.560,0.085714,0.232558
3,1394,0.100052,0,0,12,2,15,1,10,38,...,0.640430,0.260870,0.290909,0.68,0.0,0.358974,0.645833,0.826,0.178571,0.584718
4,1387,0.100073,0,0,12,2,17,1,10,38,...,0.460362,0.086957,0.254545,0.48,0.0,0.153846,0.520833,0.773,0.121429,0.411960


In [80]:
df_game_profile_train.to_pickle('df_game_profile_train.pickle')
df_game_profile_test.to_pickle('df_game_profile_test.pickle')